*for more information, checkout* [Github](https://github.com/Crazytoph/2Dcorrelation "my github page").

**CD Measurements - Data Analysis**

# DNA Origami 

## *setting up the Notebook*

In [1]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget
        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import Output
import matplotlib
import openpyxl
import re

from scipy import integrate
import lmfit
import os

In [2]:
import analise as ana
import cdata 
import hotznplots as plot

<div class="alert alert-block alert-info">
<b>Info:</b> Print always whole DataFrames
</div>

In [3]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [4]:
# change plot size
plt.rcParams["figure.figsize"] = (10,7)

"""# activate Latex, change font type
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.serif": ["Helvetica"],
})"""

'# activate Latex, change font type\nplt.rcParams.update({\n    "text.usetex": True,\n    "font.family": "sans-serif",\n    "font.serif": ["Helvetica"],\n})'

<div class="alert alert-block alert-info">
<b>Info:</b> Get the data
</div>

In [5]:
# Path for Daniel:
#path = "F:\\HZDR\\CD_data"

# My Path:
path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Data\\CD_data\\DNA Origami"
datalist = os.listdir(path)
print(datalist)

['46_forward', '46_reversed', 'Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_1M', 'GdmCl_2M', 'GdmCl_2M_20nM Origami', 'GdmCl_2M_24h', 'GdmCl_2M_80nM Origami', 'GdmCl_4M', 'GdmCl_6M', 'GdmCl_6M_alt', 'GdmSCN_0.5M', 'GdmSCN_2M', 'Urea_2M']


In [6]:
# all data in one dictionary with foldernames as names
data_all = {}
for i in range(len(datalist)):
    data_all[datalist[i]] = cdata.CData(os.path.join(path, datalist[i]))

In [7]:
# options to acces data
print(data_all['Control_0M'])
# multiple options
print(list(map(data_all.get, ['Control_0M', 'Gdm2SO4_4M'])))
print(tuple(data_all.values())[0:4])

[<cdata.CData object at 0x00000115FA1DE190>, <cdata.CData object at 0x00000115FA1E90D0>]
(<cdata.CData object at 0x00000115FF3414C0>, <cdata.CData object at 0x00000115FF324EE0>, <cdata.CData object at 0x00000115FA1DE190>, <cdata.CData object at 0x00000115FF341E80>)


## PCA

### Print-out for PCA


In [1]:
print_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Data\\PCA"

In [2]:
liste = [ 'GdmCl_4M']

In [3]:

i= 1

for key in liste:
    for col in data_all[key].cd_df.columns:
        if col >= 20  and col <= 90:
            name =  print_path + '\\' + 'GdmCl_4M' + '\\' + str(i) + '.dat'
            data_all[key].cd_df.loc[:, col].to_csv(name, sep = " ", header=False)
            i = i + 1
    print(i)

NameError: name 'data_all' is not defined

In [ ]:
print(data_all["Control_0M"].t_list,"\n", data_all["GdmCl_0.5M"].t_list, "\n", data_all["GdmCl_2M"].t_list, "\n", data_all["GdmCl_4M"].t_list, "\n", data_all["GdmCl_6M"].t_list, "\n", melt_T)

### Eigenvector output


In [48]:
print_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Data\\PCA"

In [58]:
liste = ['Control_0M']

In [111]:
for  key in liste:
    # create DataFrame for plot
    plot_df = pd.DataFrame(index=data_all[key].cd_df.loc[215:300].index)
    
    # number of components
    components = 16
    
    # read files 
    for i in range(components):
        
        name = 'eigenvector' + str(i+1) 
        # get data path
        data_path = os.path.join(print_path, key, name + '.dat')
        
        # open file and create data list
        file = open(data_path, "r")
        data = []
        
        # list of lines, of which all elemnts equal to space or enter are used as split, and then empty elements are deleted
        list_of_lines = file.readlines()
        for k in range(len(list_of_lines)):
            data.extend(re.split(' |\n', list_of_lines[k]))
        data1 = np.array(list(filter(lambda x: x != '', data)))
        
        # data list reshaped as 2D array and added to plot
        data1 = data1.reshape(86, 2)
        plot_df[i+1] = data1[:, 1]
        
    # same procedure for eigenvalues to norm eigenvectors    
    name = 'eigenvalues'
    data_path = os.path.join(print_path, key, name + '.dat')
    # open file and create data list
    file = open(data_path, "r")
    data = []
    # list of lines, of which all elemnts equal to space or enter are used as split, and then empty elements are deleted
    list_of_lines = file.readlines()
    for k in range(len(list_of_lines)):
        data.extend(re.split(' |\n', list_of_lines[k]))
        data1 = np.array(list(filter(lambda x: x != '', data)))
    
    # data list reshaped as 2D array and added to plot
    data1 = data1.reshape(16, 2)
    eigenvectors = data1
    
    # change dtype to float
    plot_df = plot_df.astype('float64')
    eigenvectors = eigenvectors.astype('float64')
    #plot_df = plot_df.multiply(eigenvectors.T[1])
    # norm between 0 - 1 if wanted
    #plot_df = plot_df.subtract(plot_df.min()).divide((plot_df.max()-plot_df.min()))
    plot.mult_func([1,2, 3, 4, 5, 6], [plot_df.T], linestyle=["solid","solid",  'dashed','dashed','dotted', 'dotted'], marker=["","","","","","",""], subtitle=[""], 
                   x_label="wavelength \ nm", y_label="eigenvectors")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Varimax Output

In [48]:
print_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Data\\PCA"

In [49]:
files = os.listdir(print_path)
print(files)

['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'Gdm2SO4_all', 'GdmCl_0.5M', 'GdmCl_1M', 'GdmCl_2M', 'GdmCl_4M', 'GdmCl_6M', 'GdmCl_all', 'GdmCl_all_lowTemp']


In [50]:
liste = ['Gdm2SO4_2M']

In [51]:
for  key in liste:
    # create DataFrame for plot
    plot_df = pd.DataFrame(index=data_all[key].t_list[:-3])
    
    # number of components
    components = 4
    
    # read files 
    for i in range(components):
        
        name = 'VARIMAX_loading_component' + str(i+1) 
        # get data path
        data_path = os.path.join(print_path, key, name + '.dat')
        
        # open file and create data list
        file = open(data_path, "r")
        data = []
        
        # list of lines, of which all elemnts equal to space or enter are used as split, and then empty elements are deleted
        list_of_lines = file.readlines()
        for k in range(len(list_of_lines)):
            data.extend(re.split(' |\n', list_of_lines[k]))
        data = np.array(list(filter(lambda x: x != '', data)))
        
        # data list reshaped as 2D array and added to plot
        data = data.reshape(len(data_all[key].t_list[:-3]), 2)
        plot_df[i+1] = data[:, 1]
   
    # change dtype to float
    plot_df = plot_df.astype('float64')
    # norm between 0 - 1 if wanted
    plot_df = plot_df.subtract(plot_df.min()).divide((plot_df.max()-plot_df.min()))
    plot.mult_func([1, 2, 3, 4], [plot_df.T], linestyle=["solid","dashdot","dashed","dotted"], marker=["o","o","o","o"], subtitle=[key], 
                   x_label="Temperature [°C]", y_label="Abstract Concentration after Varimax Rotation")

ValueError: cannot reshape array of size 34 into shape (12,2)

In [39]:
print(data_all[key].t_list, melt_T[key])
print(data_all[key].t_list[2:])

[5, 10, 20, 30, 40, 45, 50, 53, 56, 58, 60, 62, 64, 67, 70, 75, 80, 90] 65.7219345117269
[20, 30, 40, 45, 50, 53, 56, 58, 60, 62, 64, 67, 70, 75, 80, 90]


### ITTFA Output

In [8]:
print_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Data\\PCA"

In [18]:
key_list = ['Gdm2SO4_2M']
filename = 'Gdm2SO4_2M'
wave_min = 215
wave_max = 300 

t_min = 20
t_max = 90
t_list = []

for key in key_list:
    t_list.extend(data_all[key].cd_df.loc[wave_min:wave_max,t_min:t_max].columns)
    #t_list.insert(0, 10)
    #t_list.extend([100, 110])

# create DataFrame for plot
conc_df = pd.DataFrame(index=t_list)
spec_df = pd.DataFrame(index=data_all[key].cd_df.loc[wave_min:wave_max,t_min:t_max].index)

# number of components
components = 4
    
# read files, 1st rel_conectration, 2nd spectra of components and third error in rel concentrations
for i in range(components):
    name1 = 'rel_concentration_component' + str(i+1) 
    name2 = 'component' + str(i+1) 
    
    # get data path
    data_path1 = os.path.join(print_path, filename, name1 + '.dat')
    data_path2 = os.path.join(print_path, filename, name2 + '.dat')
        
    # open file and create data list
    file1 = open(data_path1, "r")
    file2 = open(data_path2, "r")
    data1 = []
    data2 = []
        
    # list of lines, of which all elemnts equal to space or enter are used as split, and then empty elements are deleted
    list_of_lines1 = file1.readlines()
    list_of_lines2 = file2.readlines()
    
    for k in range(len(list_of_lines1)):
        data1.extend(re.split(' |\n', list_of_lines1[k]))
    data1 = np.array(list(filter(lambda x: x != '', data1)))
    
    for j in range(len(list_of_lines2)):
        data2.extend(re.split(' |\n', list_of_lines2[j]))
    data2 = np.array(list(filter(lambda x: x != '', data2)))
    
    # data list reshaped as 2D array and added to plot
    data1 = data1.reshape(len(t_list), 2)
    conc_df[i+1] = data1[:, 1]
    
    data2 = data2.reshape(len(spec_df.index), 2)
    spec_df[i+1] = data2[:, 1]

# read relative errors of concentrations
name3 = 'error_in_rel_concentrations'
data_path3 = os.path.join(print_path, filename, name3 + '.dat')
file3 = open(data_path3, "r")
data3 = []
list_of_lines3 = file3.readlines()
for j in range(len(list_of_lines3)):
    data3.extend(re.split(' |\n', list_of_lines3[j]))
data3 = np.array(list(filter(lambda x: x != '', data3)))
data3 = data3.reshape(components, 2)



# change dtype to float
conc_df = conc_df.astype('float64')
spec_df = spec_df.astype('float64')

# trying to calculate error
data_measured = data_all[key].cd_df.loc[wave_min:wave_max,t_min:t_max]
data_calculated = pd.DataFrame(np.matmul(spec_df.to_numpy(), conc_df.to_numpy().T), index=data_measured.index, columns=t_list)

# renorm data
data_measured = data_measured.divide(data_measured.max(axis=0) - data_measured.min(axis=0))
data_calculated  = data_calculated.divide(data_calculated.max(axis=0) - data_calculated.min(axis=0))
df_error = np.sqrt(np.square(data_measured.subtract(data_calculated.loc[:,20:90])).sum(axis=0)) / (wave_max - wave_min - 1)
#print(df_error.max())
# plot.mult_func([20], [data_measured, data_calculated], swap=True)  # plot both together

# negative reverse and amplitude adaption
#conc_df[2] = conc_df[2] * -1
#spec_df[2] = spec_df[2] * -1
#conc_df = conc_df * (data_all[key].cd_df.loc[247, 20]/spec_df.loc[247,1])
#spec_df = spec_df * (data_all[key].cd_df.loc[247, 20]/spec_df.loc[247,1])
# norm between 0 - 1 if wanted
#conc_df = conc_df.subtract(conc_df.min()).divide((conc_df.max()-conc_df.min()))

plot.mult_func(np.arange(1, components + 1), [conc_df.T], linestyle=["solid","dashdot","dashed","dotted"], subtitle=[filename], 
                x_label="Temperature [°C]", y_label="Relative Concentration [ %]", 
               label=[*np.arange(1, components + 1), "Melting Temperature"], title="Concentration of Components after ITTFA",
               marker=["o", "X", "s"], 
               baseline=True, 
               vertical_line=[melt_T[filename]]
              )    
    
plot.mult_func(np.arange(1, components + 1), [spec_df.T], linestyle=["solid","dashdot","dashed","dotted"], marker=["","","",""], subtitle=[filename], 
                x_label="Wavelength [nm]", title="Spectra of Components from ITT", baseline=True, label=np.arange(1, components + 1),
               y_max = [spec_df.T.loc[components].max().max()], y_min = [spec_df.T.loc[components].min().min()], y_scaling=False )    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
plot.mult_func([1,3,2], [spec_df.T], marker=["","","",""], subtitle=[''], 
                x_label="wavelength /nm", baseline=True, label=['S1','S2\'', 'S3'], y_label='abstract ellipticity',
              colors=['tab:red', 'tab:grey', 'tab:green'])  

C:\Users\crazy\2Dcorrelation\2Dcorrelation\hotznplots.py:361: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(facecolor=backgroundcolor)  # create figure


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Recalculation and TS-Plots

In [19]:
df = pd.DataFrame(np.matmul(spec_df.to_numpy(), conc_df.to_numpy().T), index=data_measured.index, columns=t_list)
max_cd_value = df.abs().max().max()
sync,  assync = ana.perturbation_moving_window(df, window_size=3)
plot.heatmap(sync, x_min=[215], x_max=[300], x_label='temperature / °C', y_label='wavelength / nm',
                 c_label="",c_min=[-1e8], c_max=[1e8], title="", subtitle=[''],
                contour_lines=[-0.7*max_cd_value, -0.4*max_cd_value,  -0.2*max_cd_value,  0, 0.2*max_cd_value, 
                                0.4*max_cd_value, 0.7*max_cd_value],
            hline=melt_T[filename])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
print(data3, df_error)

In [ ]:
print(data_all['GdmCl_1M'].t_list)

*write numbers into excel*

In [20]:
excel_path = print_path + "\\" + filename + "\\output.xlsx"
print(excel_path)

C:\Users\crazy\Documents\Uni\Masterarbeit\Projekt\Data\PCA\Control_0M\output.xlsx


In [21]:
with pd.ExcelWriter(excel_path, mode='w') as writer:  
    conc_df.round(6).to_excel(writer, sheet_name="concentration ")
    
with pd.ExcelWriter(excel_path, mode='a') as writer:  
    spec_df.round(6).to_excel(writer, sheet_name="spectra")
            

In [43]:
plot.mult_func([2, 90], [data_measured.T, data_calculated.T], subtitle=['calculated vs measured spectra of 2nd component'], label=['measured', 'calculated'], x_label='Wavelength / nm')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plots

### Heat Maps

In [11]:
liste =  ["Control_0M", 'GdmCl_0.5M', 'GdmCl_1M', 'GdmCl_2M', 'GdmCl_4M', 'GdmCl_6M', 'Gdm2SO4_0.5M','Gdm2SO4_1M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M']

In [13]:
for key in liste:
    df = data_all[key].cd_df.loc[215:300,20:]
    max_cd_value = df.abs().max().max()
    plot.heatmap(df, subtitle=[""], title="", x_min=[215], x_max=[300], swap=True,
                 x_label=r"temperature $ / \ ^{\circ}C$", y_label=r"wavelength $/ \ nm$", c_label=r"molar ellipticity "
                 r"$ / \ \frac{deg \times cm^{2}}{dmol}$" , 
                 contour_lines=[-0.8*max_cd_value, -0.5*max_cd_value,  -0.25*max_cd_value,  0, 0.25*max_cd_value, 0.5*max_cd_value, 0.8*max_cd_value]
                )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\crazy\2Dcorrelation\2Dcorrelation\hotznplots.py:84: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(facecolor=backgroundcolor)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
for key in data_all.keys():
    plot.heatmap(data_all[key].cd_df, subtitle=[""], title=str(key), x_min=[215], swap=True, y_label="Temperature [°C]", x_label="Wavelength [nm]")

In [ ]:
#plot.heatmap(*tuple(data_all.values()), subtitle=tuple(data_all.keys()), title="Cd-Values", x_min=[220]*len(data_all), swap=False)

### 3D surface plots

In [8]:
df = data_all["Control_0M"].cd_df.loc[215:300]

plot.function3d(df, title="", z_label=r"molar CD \ "
                 r"$ \frac{deg \times cm^{2}}{dmol}$",
               x_label='wavelength \ nm', y_label='temperature \ °C')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


###  Function Plots 

#### CD-Spectra Plot

In [41]:
df =  data_all["Control_0M"].cd_df

plot.mult_func([20], [df], swap=True, x_min=215, x_max=300, marker=[''], subtitle=[''], baseline=True, colors=['tab:red'], x_label="wavelength / nm", label=['20 °C'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
wave_min = 215
wave_max = 330

plot.mult_func([230, 247, 275], [data_all['GdmCl_1M'].cd_df.loc[wave_min:wave_max]], 
               swap=False, x_label="Wavelength [nm]", subtitle=[""], 
               marker=['', '', '', '', '', ''], linestyle=['-', ':', ':', '', '-'], baseline=True, 
               label=['230','247','275'])

C:\Users\crazy\2Dcorrelation\2Dcorrelation\hotznplots.py:361: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(facecolor=backgroundcolor)  # create figure


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
wave_min = 215
wave_max = 300

key_list = ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M' ,'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_1M', 'GdmCl_2M',  'GdmCl_4M', 'GdmCl_6M']
cd_liste = []
for key in key_list:
    cd_liste.append(data_all[key].cd_df)
    
plot.mult_func([20], cd_liste, title=key, swap=True, x_min=wave_min, x_max=wave_max,
                   x_label="Wavelength [nm]", y_label="CD values [mdeg]", subtitle=[""],  baseline=True, label=key_list,
               marker=[""]*len(key_list)
                  )



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
liste = np.zeros(len(key_list))
for i in range(len(key_list)):
    liste[i] = cd_liste[i].loc[220,20]
print(np.mean(liste), np.std(liste), np.mean(liste)/np.std(liste))

68548120.26446313 19127675.77705771 3.5837140415502926


#### Max/Min Plots


multiple probes

In [ ]:
excel_path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\output.xlsx"

In [24]:
# liste 
liste =  ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M',  'GdmCl_0.5M', 'GdmCl_2M', 'GdmCl_2M_24h' , 'GdmCl_4M']
wave_min = 260
wave_max =  280

for key in liste:
    plot_data =  ana.min_wave(data_all[key].cd_df.abs(), wave_min=wave_min, wave_max=wave_max)
    plot_data = plot_data.drop(['Value'], axis=1)   
    plot_data.rename(columns = {'Wavelength': key}, inplace = True)
    with pd.ExcelWriter(excel_path, mode='a') as writer:  
                    plot_data.to_excel(writer, sheet_name=key)

NameError: name 'excel_path' is not defined

In [50]:
wave_min = 260
wave_max =  280
key = "Gdm2SO4_4M"

# First data you want to add to the plot
plot_data =  ana.max_wave(data_all[key].cd_df, wave_min=wave_min, wave_max=wave_max)
plot_data = plot_data.drop(['Wavelength'], axis=1)   
plot_data.rename(columns = {'Value': key}, inplace = True)



# other Data you want to add, if always same max/min values then loop would be possible
#plot_data["GdmCl_4M"] = ana.max_wave(data_all["GdmCl_4M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]
#plot_data["Gdm2SO4_2M"] = ana.max_wave(data_all["Gdm2SO4_2M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]
#plot_data["GdmSCN_2M"] 2= ana.max_wave(data_all["GdmSCN_2M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]


plot.mult_func([key], [plot_data.T], subtitle=[""], title="Max. CD Value between " + str(wave_min) + 'nm - ' +str(wave_max) + " nm")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

one probe

In [ ]:
wave_min = 260
wave_max =  290
title = "Max. CD-Value between " + str(wave_min) + "nm - " + str(wave_max) +"nm"

for key in data_all.keys():
    plot_data =  ana.max_wave(data_all[key].cd_df, wave_min=wave_min, wave_max=wave_max)
    plot.mult_func(["Value"], [plot_data.T], subtitle=[""], vertical_line=[melt_T[key]], title=title, label=[key, "melting Temp."])



## Correlation Analyis

<div class="alert alert-block alert-info">
<b>Info:</b> Noda, I. (2007). Two-dimensional correlation analysis useful for spectroscopy, chromatography, and other analytical measurements. Analytical Sciences, 23(2), 139–146. https://doi.org/10.2116/analsci.23.139
</div>

<div class="alert alert-block alert-info">
<b>Info:</b> Noda, I. (2015). Techniques of two-dimensional (2D) correlation spectroscopy useful in life science research. Biomedical Spectroscopy and Imaging, 4(2), 109–127. https://doi.org/10.3233/bsi-150105
</div>

### Homogenous Spectrum

*optional: pareto scaling, auto scaling and Reference and projection*

In [10]:
# list of measurements on which correlation should be performed
liste =  ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M','GdmCl_1M', 'GdmCl_2M' , 'GdmCl_4M', 'GdmCl_6M']
#'Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_2M' , 'GdmCl_4M', 'GdmCl_6M'
# min and max temperature between which correlations should be analysed
t_min = 20
t_max = 90

for key in liste:
    # get reference
    ref = data_all[key].cd_df.loc[215:300, 20]
    
    # sort out values with HT above 900
    df = data_all[key].cd_df.loc[215:300, :t_max]
    
    sync, assync =  ana.correlation(df, ref_spec=[ref], scaling='pareto',  center=False)
    for i in df.index:
        for j in df.index:
            if sync.loc[i,j] == 0:
                sync.loc[i, j] = 1
    plot_df = assync.divide(sync)

    plot.heatmap(plot_df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# list of measurements on which correlation should be performed
liste =  liste =  ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M','GdmCl_1M', 'GdmCl_2M' , 'GdmCl_4M', 'GdmCl_6M']
#'Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_2M' , 'GdmCl_4M', 'GdmCl_6M'
# min and max temperature between which correlations should be analysed
t_min = 20
t_max = 90

# collection of sync and assync plots
sync_dic = {}
async_dic ={}

# max scaling values for upscaling correlation values with absorption spectrum
max_scaling_value_sync = 0
max_scaling_value_async = 0

In [13]:
# define witdh of additionaly absorbtion correlation band
width = 5

for key in liste:
    # get reference
    ref = data_all[key].cd_df.loc[215:300, 20]
    
    # sort out values with HT above 900
    df = data_all[key].cd_df.loc[215:300, t_min:t_max]
    
    # add absorbance as extra row 
    for i in range(width):
        df = df.append(data_all[key].absorb_df.loc[260, t_min:t_max], ignore_index = True)
        ref[301+i] = data_all[key].absorb_df.loc[260, 20]
    ref.index = np.arange(86 + width)
    
    # calculate
    sync, assync = ana.correlation(df, ref_spec=[ref], center=False)
    sync1, assync1 = ana.correlation(data_all[key].cd_df.loc[215:300, t_min:t_max], ref_spec=[data_all[key].cd_df.loc[215:300, 20]], scaling='pareto', center=False)
        
    # pareto scaling for CD-part
    sync.loc[:85, :85] = sync1.to_numpy()
    assync.loc[:85, :85] = assync1.to_numpy()
    
    # get the sacling the absorption values for sync, and assync and compare with max scaling value so far...
    max_cd_value_sync = sync.abs().max().max()
    max_abs_value_sync = sync.loc[86:].abs().max().max()
    max_scaling_value_sync = (max_cd_value_sync / max_abs_value_sync)
        
    max_cd_value_async = assync.abs().max().max()
    max_abs_value_async = assync.loc[86:].abs().max().max()
    max_scaling_value_async = (max_cd_value_async / max_abs_value_async)
        
    print(max_scaling_value_sync, max_scaling_value_async)
    # adding DataFrame to plot dics
    sync_dic[key] = sync
    async_dic[key] =  assync

21.655153097330512 19.74706886424258
17.620500066410067 13.078106829060031
21.681512325188503 17.851505419785063
15.413401116773587 18.000943634882447
15.670141784593056 14.60663529752312
41.07190140273921 57.82619563211382
18.813384065976916 16.17582137376178
29.126890785532215 27.655453475152143
18.931900180838223 17.903615828818012
15.758076205693355 13.457740214862396
29.33075923347965 23.182616496617985


In [14]:
# max scaling values for upscaling correlation values with absorption spectrum
max_scaling_value_sync = 21.655153097330512
max_scaling_value_async = 19.74706886424258

In [15]:
for key in sync_dic.keys():
    max_cd_value = sync_dic[key].abs().max().max()
    sync_dic[key].loc[86:] =  sync_dic[key].loc[86:] * max_scaling_value_sync
    sync_dic[key].loc[:85,86:] = sync_dic[key].loc[:85,86:] * max_scaling_value_sync
    plot.heatmap(sync_dic[key],  title='', subtitle=["" ], y_label="Wavelength 1 [nm]", 
                 x_label="Wavelength 2 [nm]", c_label="", c_min=[-max_cd_value], c_max=[max_cd_value], hline=85, vline=85,
                 contour_lines=[-0.8*max_cd_value,-0.5*max_cd_value, -0.3*max_cd_value, -0.15*max_cd_value, 0, 0.15*max_cd_value, 0.3*max_cd_value, 0.5*max_cd_value, 0.8*max_cd_value])
    
    max_cd_value = async_dic[key].abs().max().max()
    async_dic[key].loc[86:] =  async_dic[key].loc[86:] * max_scaling_value_async
    async_dic[key].loc[:85,86:] = async_dic[key].loc[:85,86:] * max_scaling_value_async
    plot.heatmap( async_dic[key], title="", subtitle=[ "" ], y_label="Wavelength 1 [nm]", 
                 x_label="Wavelength 2 [nm]",  c_label="", hline=85, vline=85,
                 contour_lines=[-0.8*max_cd_value, -0.5*max_cd_value,  -0.3*max_cd_value, -0.15*max_cd_value, 0, 0.15*max_cd_value, 0.3*max_cd_value, 
                                0.5*max_cd_value, 0.8*max_cd_value] )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Tools for Analysis

<div class="alert alert-block alert-info">
<b>Info:</b> Change 'key' to wanted measurement!
</div>

In [ ]:
key1 = "GdmCl_2M"
key2 = "GdmCl_2M_24h"
ref1 = data_all[key1].cd_df.loc[:, 20]
ref2 = data_all[key2].cd_df.loc[:, 20]
sync, assync = ana.correlation(data_all[key1].cd_df.loc[:, :45], data_all[key2].cd_df.loc[:, :45], scaling='pareto', ref_spec=[ref1, ref2], center=False)

In [ ]:
plot.heatmap(sync, assync,  title=str(key1) + str(key2), subtitle=["Synchronous Spectrum", "Asynchronous Spectrum"], x_min=[210, 210], y_min=[210, 210],
                y_label= str(key1) + " WL[nm]", x_label= str(key2) + " WL[nm]")

**Diagonal Analysis**

In [ ]:
arr = sync.to_numpy()
diag =  pd.DataFrame(arr.diagonal(), index=sync.index)
wave =  list(sync.index)[:-1]
diff =  pd.DataFrame(np.diff(arr.diagonal()), index=wave)
plot.mult_func([0], [diff.T, diag.T], baseline=True, x_label="wavelength [nm]", subtitle=["Diagonal with differential"], label=["CD Values", "Differential"], x_min=210)

**Line Analyis**

In [ ]:
wavelength=247
# adjust wavelength to shift line analysis*

In [ ]:
arr = sync.loc[wavelength, :].to_numpy()
df = pd.DataFrame(arr, index=sync.index).T
diff = pd.DataFrame(np.diff(arr), index=list(sync.index)[:-1]).T
plot.mult_func([0], [df, diff], x_label="wavelength [nm]", baseline=True, x_min=210, subtitle=["Synchronous Line Analysis"], label=[wavelength, "derivative"])

**Excel**

In [106]:
excel_path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\output.xlsx"

create new document

In [109]:
with pd.ExcelWriter(excel_path, mode='w') as writer:  

                    comp2_conc_df.round(2).to_excel(writer, sheet_name="output ")

Homgeneous Spectra Values

In [ ]:
# liste 
liste =  ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_2M', 'GdmCl_2M_24h' , 'GdmCl_4M', 'GdmCl_6M', 'Urea_2M']
t_min = 20
t_max = 90



for key in liste:
    # get reference
    ref = data_all[key].cd_df.loc[:, 20]
    # sort out values with HT above 900
    df = data_all[key].cd_df
    # calculate
    sync1, assync1 = ana.correlation(df.loc[:,:45], scaling='pareto', ref_spec=[ref], center=False)
    df1 = sync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
    df2 = assync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
    with pd.ExcelWriter(excel_path, mode='a') as writer:  
                    df1.to_excel(writer, sheet_name=key  + " sync")
                    df2.to_excel(writer, sheet_name=key  + " async")

Heterogeneous Spectra Values

In [ ]:
# Liste

#plot_list = ["Control_0M", 'GdmCl_0.5M', 'GdmCl_2M',  'GdmCl_4M', 'GdmCl_6M']
plot_list = ["Control_0M", 'Gdm2SO4_0.5M', 'Gdm2SO4_2M',  'Gdm2SO4_4M', 'Gdm2SO4_6M']
anzahl = len(plot_list)
t_min = 20 
t_max = 90


# calculation 
for i in range(anzahl):
    for j in range(anzahl):
        if i >= j:
            continue
        df1 = data_all[plot_list[i]].cd_df
        df2 = data_all[plot_list[j]].cd_df
        ref1 = df1.loc[:, 20]
        ref2 = df2.loc[:, 20]
        sync1, assync1 = ana.correlation(df1.loc[:, :t_max], df2.loc[:, :t_max], scaling='pareto', ref_spec=[ref1, ref2], center=False)
        df1 = sync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
        df2 = assync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
        with pd.ExcelWriter(excel_path, mode='a') as writer:  
            df1.to_excel(writer, sheet_name=plot_list[i] + " vs.  " + plot_list[j]  + " sync")
            df2.to_excel(writer, sheet_name=plot_list[i] + " vs.  " + plot_list[j]  + " async")
            
# extra calculation for 2M vs 2M_24h            
"""df1 = data_all["GdmCl_2M"].cd_df
df2 = data_all["GdmCl_2M_24h"].cd_df
ref1 = df1.loc[:, 20]
ref2 = df2.loc[:, 20]
sync1, assync1 = ana.correlation(df1.loc[:, :t_max], df2.loc[:, :t_max], scaling='pareto', ref_spec=[ref1, ref2], center=False)
df1 = sync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
df2 = assync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
with pd.ExcelWriter(excel_path, mode='a') as writer:  
    df1.to_excel(writer, sheet_name="2M vs. 2M 24h sync")
    df2.to_excel(writer, sheet_name="2M vs. 2M 24h async")"""

In [ ]:
arr = assync.loc[wavelength, :].to_numpy()
df = pd.DataFrame(arr, index=assync.index).T
diff = pd.DataFrame(np.diff(arr), index=list(assync.index)[:-1]).T
plot.mult_func([0], [df, diff], x_label="wavelength [nm]", baseline=True, x_min=210, subtitle=["synchronous Line Analysis"])

### Heterogenous Spectrum

*with pareto scaling and Control Measurement as Reference*

In [ ]:
print(data_all.keys())

In [ ]:
#plot_list = ["Control_0M", 'GdmCl_0.5M', 'GdmCl_2M',  'GdmCl_4M', 'GdmCl_6M']
plot_list = ["Control_0M", 'Gdm2SO4_0.5M', 'Gdm2SO4_2M',  'Gdm2SO4_4M', 'Gdm2SO4_6M']
anzahl = len(plot_list)
t_min = 20 
t_max = 90

In [ ]:
for i in range(anzahl):
    for j in range(anzahl):
        if i >= j:
            continue
        title = "Heterogenoues 2D Correlation between " + plot_list[i] + " " + plot_list[j]
        df1 = data_all[plot_list[i]].cd_df
        df2 = data_all[plot_list[j]].cd_df
        ref1 = df1.loc[:, 20]
        ref2 = df2.loc[:, 20]
        sync1, assync1 = ana.correlation(df1, df2, scaling='pareto', ref_spec=[ref1, ref2], center=False)
        sync2, assync2 = ana.correlation(df1, df2, scaling='auto', ref_spec=[ref1, ref2], center=False)
        plot.heatmap(sync1, assync1, title=title, subtitle=["Synchronous Spectrum", "Asynchronous Spectrum", "Synchronous Spectrum", "Asynchronous Spectrum"], x_min=[210, 210, 210, 210], y_min=[210, 210, 210, 210],
                y_label= plot_list[i] + " WL[nm]", x_label= plot_list[j] + " WL[nm]")

### Perturbation-Correlation Moving-Window 2D Correlation Spectroscopy

*different window size can be choosen*

In [18]:
liste =  ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M', 'Gdm2SO4_2M', 'GdmCl_1M' ,  'GdmCl_2M' , 'GdmCl_4M']

In [19]:
for key in liste:
    df = data_all[key].cd_df.loc[215:300, 20:]
    max_cd_value = df.abs().max().max()
    sync, assync, = ana.perturbation_moving_window(df, window_size=3)
    plot.heatmap(sync, x_min=[215], x_max=[300], 
                 c_label="",c_min=[-0.8e8], c_max=[0.8e8], title="", subtitle=[''],
                contour_lines=[-0.7*max_cd_value, -0.4*max_cd_value,  -0.2*max_cd_value,  0, 0.2*max_cd_value, 
                                0.4*max_cd_value, 0.7*max_cd_value],
                hline=melt_T[key])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2D Correlation of Null Space Projection

*different projection_vectors/matrices can be choosen*

In [ ]:
liste =  ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', "Gdm2SO4_6M", "Control_0M" ,'GdmCl_0.5M', 'GdmCl_2M' , 'GdmCl_4M', 'GdmCl_6M' ]

In [ ]:
for key in liste:
    # get reference
    ref = data_all[key].cd_df.loc[:, 20]
    
    # create dynamic spectrum! and calculate projection
    df = ana.projection_matrix(data_all[key].cd_df.subtract(ref, axis=0), row_min_max=[247], center=False)
    plot.mult_func([264, 274, 281], [df], subtitle=[key])

In [ ]:
for key in liste:
    # get reference
    ref = data_all[key].cd_df.loc[:, 20]
    
    # create dynamic spectrum! and calculate projection
    df = ana.projection_matrix(data_all[key].cd_df.subtract(ref, axis=0), row_min_max=[247], center=False)
    sync, assync = ana.correlation(df)
            
    plot.heatmap(sync, assync, x_min=[220, 220], y_min=[220,220], subtitle=[""], title=str(key), c_max=[2e+20, 2e+20], c_min=[-1e+20,-1e+20])

In [ ]:
for key in liste:
    # get reference
    ref = data_all[key].cd_df.loc[:, 20]
    
    # create dynamic spectrum! and calculate projection
    df = ana.projection_matrix(data_all[key].cd_df.subtract(ref, axis=0), row_min_max=[247], center=False)
    
     # sort out values with HT above 900
    for wave in data_all[key].ht_df.index:
        if data_all[key].ht_df.max(axis=1).loc[wave] >= 900:
            df = df.drop(wave)
    # calculate
    sync1, assync1, = ana.correlation(df, ref_spec=[0])
    # plot
    plot.heatmap(sync1, assync1, title=str(key), subtitle=["Synchronous Spectrum", "Asynchronous Spectrum"], y_label="wavelength [nm]", 
                 x_label="wavenlength [nm]", x_min=[220, 220], y_min=[220,220])

## Sigmoid Fit and Melting Temperature

In [45]:
graph={}
melt_T={}
for key in data_all.keys():
    fit, params, error = ana.sigmoid_fit(ana.normalize(data_all[key].cd_df))
    x =  np.arange(20, 91, 0.1)
    deriv = pd.DataFrame(ana.sigmoid_deriv(x, *params), index=x, columns=["deriv"]).T
    maximum = deriv.max(axis=1) 
    T = float(deriv.idxmax(axis=1))
    maxi = pd.DataFrame({T:1}, index=["Maximum"])
    deriv = deriv.div(maximum, axis=0)
    fit = pd.concat([fit, deriv], axis=0)
    fit = pd.concat([fit, maxi], axis=0)
    graph[key] = [fit]  
    melt_T[key] = T
plot.mult_func([247, "fit", "deriv", "Maximum"], *graph.values(),  title="Sigmoid-Fit and Derivative(adjusted)",
               subtitle=list(graph.keys()), marker=['x', '', '', 'x'], linestyle=["","-", ":",""])

[3.29217681e-02 5.00000000e+01] [ 0.03488354 20.73946015]
[2.50839282e-02 5.00000000e+01] [ 0.03466112 27.90790904]
[ 0.18677066 61.81964508] [0.01970739 0.53616891]
[ 0.24733276 63.42770043] [0.02193106 0.35106698]
[ 0.21601553 62.80485806] [0.02232408 0.46864971]
[ 0.3        63.95213937] [0.02830562 0.31740657]
[ 0.28173998 64.87467936] [0.02577176 0.3342244 ]
[ 0.16713782 59.57322067] [0.02171953 0.7254048 ]
[ 0.2364997  63.66483834] [0.01583726 0.27656016]
[ 0.13151325 61.12561838] [0.01504661 0.77071133]
[ 0.2939166  61.61989282] [0.01911078 0.21232322]
[ 0.23037046 61.81760758] [0.03009739 0.54504503]
[ 0.26911012 63.19314219] [0.02238773 0.30450951]
[ 0.2792924  63.37278365] [0.02065513 0.28366469]
[ 0.28883307 58.5239527 ] [0.02428439 0.29303091]
[ 0.17783219 50.        ] [0.01243005 0.44683528]
[ 0.15523255 58.75746468] [0.01973834 0.7946298 ]
[1.41701333e-02 5.00000000e+01] [1.82154880e-02 2.37113819e+01]
[2.29854877e-02 7.00000000e+01] [ 0.01983874 16.65590285]
[ 0.19424656

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\crazy\2Dcorrelation\2Dcorrelation\hotznplots.py:476: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  plt.tight_layout()


Plot the melting Temperatures for the Gdm2SO4 and GdmCl

In [ ]:
x = [0., 0.5, 2., 4.]
y = list(melt_T.values())[0:4]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_2M"], melt_T["GdmCl_4M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], title="Melting Temperature", subtitle=[""], x_label="Concentration[M]", y_label="Temperature[K]")

## Absorbance Values

In [31]:

for key in data_all.keys():
    plot.heatmap(data_all[key].absorb_df, subtitle=[key], title="Absorbance Values", x_min=[215], x_max=[300], c_min=[0]*len(data_all))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
for key in data_all.keys():
    df = data_all[key].absorb_df
    deriv_df = ana.derivative(df)*10
    plot.mult_func([260], [df - df.min(axis=1).loc[260], deriv_df], subtitle=[key], linestyle=["-",":"], marker=["x",""])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
graph={}
melt_T={}
error_a = {}
error_b = {}
coop={}
elements = ["Control_0M", "Gdm2SO4_0.5M", "Gdm2SO4_2M", "Gdm2SO4_4M", "Control_0M", "GdmCl_0.5M", "GdmCl_2M", "GdmCl_4M" ]
key = "GdmSCN_0.5M"
fit, params, std = ana.sigmoid_fit(ana.normalize(data_all[key].absorb_df), wave=260, a_range=[0.2, 0.5])
x =  np.arange(20, 91, 0.1)
deriv = pd.DataFrame(ana.sigmoid_deriv(x, *params), index=x, columns=["deriv"]).T
maximum = deriv.max(axis=1) 
T = float(deriv.idxmax(axis=1))
maxi = pd.DataFrame({T:1}, index=["Maximum"])
deriv = deriv.div(maximum, axis=0)
fit = pd.concat([fit, deriv], axis=0)
fit = pd.concat([fit, maxi], axis=0)
graph[key] = [fit]  
melt_T[key] = params[1]
error_a[key] = std[0]
error_b[key] = std[1]
coop[key] = params[0] 
plot.mult_func([260, "fit", "deriv"], graph[key],  title="Sigmoid-Fit and Derivative(adjusted)", subtitle=list(graph.keys()), marker=['x', '', '', 'x'], linestyle=["","-", ":",""])

In [ ]:
graph={}
melt_T={}
error_a = {}
error_b = {}
coop={}
elements = ["Control_0M", "Gdm2SO4_0.5M", "Gdm2SO4_2M", "Gdm2SO4_4M", "Control_0M", "GdmCl_0.5M", "GdmCl_2M", "GdmCl_4M" ]
for key in data_all.keys():
    fit, params, std = ana.sigmoid_fit(ana.normalize(data_all[key].absorb_df), wave=260, a_range=[0.2, 0.5])
    x =  np.arange(20, 91, 0.1)
    deriv = pd.DataFrame(ana.sigmoid_deriv(x, *params), index=x, columns=["deriv"]).T
    maximum = deriv.max(axis=1) 
    T = float(deriv.idxmax(axis=1))
    maxi = pd.DataFrame({T:1}, index=["Maximum"])
    deriv = deriv.div(maximum, axis=0)
    fit = pd.concat([fit, deriv], axis=0)
    fit = pd.concat([fit, maxi], axis=0)
    graph[key] = [fit]  
    melt_T[key] = params[1]
    error_a[key] = std[0]
    error_b[key] = std[1]
    coop[key] = params[0] 
plot.mult_func([260, "fit", "deriv"], graph["GdmCl_4M"],  title="Sigmoid-Fit and Derivative(adjusted)", subtitle=list(graph.keys()), marker=['x', '', '', 'x'], linestyle=["","-", ":",""])

In [ ]:
x = [0., 0.5, 2., 4.]
y = list(melt_T.values())[0:4]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_2M"], melt_T["GdmCl_4M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], error=error_b, title="Melting Temperature",
               subtitle=[""], x_label="Concentration[M]", y_label="Temperature[°C]")

In [ ]:
x = [0., 0.5, 2., 4.]
y = list(coop.values())[0:4]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [coop["Control_0M"], coop["GdmCl_0.5M"], coop["GdmCl_2M"], coop["GdmCl_4M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], error=error_a, title="Variance",
               subtitle=[""], x_label="Concentration[M]", y_label="Temperature[K^2]")

### LM Fit

In [ ]:
df = data_all["Control_0M"].cd_df
plot.mult_func([280], [df], swap=False, linestyle=[""], marker=["X"], x_label="Temperature [°C]", subtitle=[""], title="Control Measurement Hyperchromatic Shift", label=["260 nm"])

In [ ]:
# Parameter
sample = "Control_0M"
melt_T = {}
error_T = {}
residual = pd.DataFrame(index=["R-Wert"])
for key in liste:
    fit, params, params_error = ana.lm_fit(data_all[key].absorb_df, guess=[0.13, 0.36, 0.4])
    melt_T[key] = params['xc']
    error_T[key] = params_error
    residual[key] = [1 - (np.sum(fit["residual"]**2)/np.sum((fit[260] - fit[260].mean(axis=0))**2))]
    plot.mult_func([260, "fit"], [fit], error={"fit": fit["error"]}, swap=True, 
                   x_label="Temperature [°C]", y_label="Absorbance [%]", label=["data", "sigmoid fit", 'melting temperature'], marker=['o', ''], linestyle=[" ", "-"],
                  subtitle=["Hyperchromatic shift"], title=key, vertical_line=[params['xc']])

In [ ]:
x = [0., 0.5, 2., 4., 6.]
y = list(melt_T.values())[0:5]

y_error = list(error_T.values())[0:5]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T
error= {"Gdm2SO4":y_error}

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_2M"], melt_T["GdmCl_4M"], melt_T["GdmCl_6M"]]
y2_error = [error_T["Control_0M"], error_T["GdmCl_0.5M"], error_T["GdmCl_2M"], error_T["GdmCl_4M"], error_T["GdmCl_6M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T
error["GdmCl"] = y2_error 

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], title="Melting Temperature", subtitle=[""], x_label="Concentration [M]", y_label="Temperature [°C]", error=error_T)

### Linear LM Fit for Melting Temperature


In [8]:
print(data_all.keys())

dict_keys(['46_forward', '46_reversed', 'Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_1M', 'GdmCl_2M', 'GdmCl_2M_20nM Origami', 'GdmCl_2M_24h', 'GdmCl_2M_80nM Origami', 'GdmCl_4M', 'GdmCl_6M', 'GdmCl_6M_alt', 'GdmSCN_0.5M', 'GdmSCN_2M', 'Urea_2M'])


In [10]:

melt_T = {}
comp2_conc = {}
liste = ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_1M', 'GdmCl_2M','GdmCl_2M_20nM Origami', 'GdmCl_2M_24h', 'GdmCl_2M_80nM Origami', 'GdmCl_4M', 'GdmCl_6M',  'GdmSCN_0.5M', 'Urea_2M']

In [11]:
for key in liste:

    # fit three secitons
    df =  data_all[key].absorb_df.loc[:,20:]
    high_fit, high_params, high_std =  ana.lm_fit(df.iloc[:,-3:], f_type='linear')
    low_fit, low_params, low_std =  ana.lm_fit(df.iloc[:,:3], f_type='linear')


    # normalize absorbance values and get next 3 values to 0.5
    df_norm = ana.normalize(df).loc[260, :]
    df_sort = pd.DataFrame(df_norm.iloc[(df_norm-0.5).abs().argsort()[:3]], columns=[260])
    # make linear fit
    middle_fit, middle_params, middle_std = ana.lm_fit(df.loc[:, df_sort.index], f_type='linear')


    # calculate cross of middle and median
    melt_T[key] = ( ( (- high_params["y0"] - low_params["y0"]) / 2 ) + middle_params["y0"] ) / ( ( (high_params["m"] + low_params["m"]) / 2 ) - middle_params["m"] ) 
    # calculate error 
    teiler = ( ( (high_params["m"] + low_params["m"]) / 2 ) - middle_params["m"] ) 
    nenner =  ( ( (- high_params["y0"] - low_params["y0"]) / 2 ) + middle_params["y0"] ) 
    t_melt_error =  np.sqrt( (-0.5 * high_std["y0"] / teiler)**2 + (-0.5 * low_std["y0"] / teiler)**2 + (middle_std["y0"] / teiler)**2
                              + (nenner * 0.5 * high_std["m"] / (teiler**2) )**2 + (nenner * 0.5 * low_std["m"] / (teiler**2) )**2 + (nenner * -1 * middle_std["m"] / (teiler**2) )**2)

    
    # make nice plot
    plot_fit= pd.DataFrame(ana.linear(df.columns, *high_params.values()), columns=["high plateu"], index=df.columns)
    plot_fit["low plateu"] = ana.linear(df.columns, *low_params.values())
    plot_fit["middle"] = ana.linear(df.columns, *middle_params.values())
    plot_fit["Median"] = 0.5*(plot_fit["high plateu"] + plot_fit["low plateu"])
    comp2_conc[key] = (df.loc[260] - plot_fit["low plateu"]) / (plot_fit["high plateu"] - plot_fit["low plateu"])

    plot.mult_func([260, "high plateu", "low plateu", "middle", "Median"], [df, plot_fit.T], 
                   vertical_line=[melt_T[key]], marker=["X", "", "", "", ""], linestyle=["--", ":", ":", "-.", "-."],
                   y_scaling=True, y_min=[df.min(axis=1).loc[260] - 0.02 ], y_max=[df.max(axis=1).loc[260] + 0.02], 
                   title=key, subtitle=[""], sec_ax=[df.min(axis=1).loc[260], df.max(axis=1).loc[260]], y_label="absorption / %",
                   x_label='temperature / °C', label=["total absorption",  "upper baseline", "lower baseline", "linear interpolation", "center line", "melting temperature"])

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 1.7907e-06
    reduced chi-square = 1.7907e-06
    Akaike info crit   = -38.9945189
    Bayesian info crit = -40.7972944
[[Variables]]
    m:   4.5476e-04 +/- 1.2389e-04 (27.24%) (init = 1)
    y0:  0.45481350 +/- 0.01014723 (2.23%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -0.997
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 3.7600e-07
    reduced chi-square = 3.7600e-07
    Akaike info crit   = -43.6768616
    Bayesian info crit = -45.4796371
[[Variables]]
    m:   9.8400e-05 +/- 4.3359e-05 (44.06%) (init = 1)
    y0:  0.35534533 +/- 0.00134809 (0.38%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m,

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 2.0782e-06
    reduced chi-square = 2.0782e-06
    Akaike info crit   = -38.5478172
    Bayesian info crit = -40.3505926
[[Variables]]
    m:   9.9431e-04 +/- 1.3347e-04 (13.42%) (init = 1)
    y0:  0.45338000 +/- 0.01093152 (2.41%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -0.997
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 3.8240e-08
    reduced chi-square = 3.8240e-08
    Akaike info crit   = -50.5339750
    Bayesian info crit = -52.3367505
[[Variables]]
    m:  -3.7950e-05 +/- 1.3828e-05 (36.44%) (init = 1)
    y0:  0.38311617 +/- 4.2992e-04 (0.11%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m,

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 1.4068e-06
    reduced chi-square = 1.4068e-06
    Akaike info crit   = -39.7183176
    Bayesian info crit = -41.5210930
[[Variables]]
    m:   6.9760e-04 +/- 1.0981e-04 (15.74%) (init = 1)
    y0:  0.43787800 +/- 0.00899409 (2.05%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -0.997
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 6.0423e-09
    reduced chi-square = 6.0423e-09
    Akaike info crit   = -56.0692821
    Bayesian info crit = -57.8720575
[[Variables]]
    m:   3.1133e-04 +/- 5.2328e-06 (1.68%) (init = 1)
    y0:  0.36192159 +/- 1.6495e-04 (0.05%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, 

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 3
    # variables        = 2
    chi-square         = 1.8115e-05
    reduced chi-square = 1.8115e-05
    Akaike info crit   = -32.0521993
    Bayesian info crit = -33.8549747
[[Variables]]
    m:   0.01084090 +/- 3.9404e-04 (3.63%) (init = 1)
    y0: -0.02179150 +/- 0.03227375 (148.10%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -0.997
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 3
    # variables        = 2
    chi-square         = 1.2161e-05
    reduced chi-square = 1.2161e-05
    Akaike info crit   = -33.2476759
    Bayesian info crit = -35.0504513
[[Variables]]
    m:   0.00367720 +/- 2.4659e-04 (6.71%) (init = 1)
    y0:  0.30347133 +/- 0.00766668 (2.53%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m,

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 2.4499e-07
    reduced chi-square = 2.4499e-07
    Akaike info crit   = -44.9619437
    Bayesian info crit = -46.7647191
[[Variables]]
    m:   7.4131e-04 +/- 4.5825e-05 (6.18%) (init = 1)
    y0:  0.44485300 +/- 0.00375328 (0.84%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -0.997
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 3
    # variables        = 2
    chi-square         = 9.5938e-06
    reduced chi-square = 9.5938e-06
    Akaike info crit   = -33.9590294
    Bayesian info crit = -35.7618048
[[Variables]]
    m:  -2.7950e-05 +/- 2.1902e-04 (783.60%) (init = 1)
    y0:  0.36149050 +/- 0.00680951 (1.88%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m,

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
print(melt_T)

{'Control_0M': 63.88046407740443, 'Gdm2SO4_0.5M': 64.52277673479793, 'Gdm2SO4_1M': 64.84295918577699, 'Gdm2SO4_2M': 64.33702771140105, 'Gdm2SO4_4M': 65.39728202507145, 'Gdm2SO4_6M': 61.39443444391862, 'GdmCl_0.5M': 64.87554508796475, 'GdmCl_1M': 65.7219345117269, 'GdmCl_2M': 62.22456104786192, 'GdmCl_2M_20nM Origami': 62.832302577436735, 'GdmCl_2M_24h': 63.688782533417864, 'GdmCl_2M_80nM Origami': 64.26286079346826, 'GdmCl_4M': 59.194612694153875, 'GdmCl_6M': 53.41120314608025, 'GdmSCN_0.5M': 58.69785533338373, 'Urea_2M': 59.00866465738076}


In [13]:
print(data_all['Gdm2SO4_2M'].cd_df.loc[247])

20   -1.705819e+08
30   -1.697162e+08
40   -1.681870e+08
45   -1.696219e+08
50   -1.621085e+08
53   -1.606941e+08
58   -1.563523e+08
60   -1.505902e+08
62   -1.364747e+08
64   -1.124725e+08
67   -7.772312e+07
70   -6.566755e+07
75   -6.092276e+07
80   -5.539280e+07
90   -5.574734e+07
Name: 247, dtype: float64


In [12]:
x = [0., 0.5, 1., 2., 4., 6.]
y =  [melt_T["Control_0M"], melt_T["Gdm2SO4_0.5M"], melt_T["Gdm2SO4_1M"], melt_T["Gdm2SO4_2M"], melt_T["Gdm2SO4_4M"], melt_T["Gdm2SO4_6M"]]

graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_1M"], melt_T["GdmCl_2M"], melt_T["GdmCl_4M"], melt_T["GdmCl_6M"]]

graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], title="Melting Temperature", subtitle=[""], 
               x_label="salt concentration / M", y_label="temperature / °C", label=[r'$Gdm_{2}SO_{4}$',  r"$GdmCl$ "],
               marker=["X","o"], linestyle=["-.", "-",], backgroundcolor='white', colors=['red',  'black'], error={"Gdm2SO4": 0.5, "GdmCl": 0.5})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Long measurements

# TD Calculations


<div class="alert alert-block alert-info">
<b>Info:</b> Steps after Mergny, J.-L., & Lacroix, L. (2003). Analysis of Thermal Melting Curves. In OLIGONUCLEOTIDES (Vol. 13). www.liebertpub.com
</div>

**Hypochromatic Shift at 260nm  wit Low and High Plateau Fit**

In [ ]:
raw = data_all["Control_0M"].absorb_df
low_fit, params_lowfit = ana.lm_fit(raw[[20,30,40]], f_type='linear', guess=[0.0001, 0.35])
high_fit, params_highfit = ana.lm_fit(raw[[70, 80,90]], f_type='linear', guess=[0.0001, 0.48])
sig_fit, params_sigfit = ana.lm_fit(raw, f_type='s1', guess=[0.2, 0.35, 0.5, 63])

**Calculating Fraction Folded with High Plateu adjusted manually**

In [ ]:
data_treated = pd.DataFrame(raw.loc[260])
data_treated["low_fit"] = ana.linear(np.array(raw.columns), *params_lowfit.values())
data_treated["high_fit"] = ana.linear(np.array(raw.columns), 0.0004495999948082116, 0.45451300041966917)
data_treated["intermediate"] = 0.5*(data_treated["low_fit"] + data_treated["high_fit"])
data_treated["fit"] = sig_fit["fit"]
data_treated["baseline_fit"] = (data_treated["high_fit"] - data_treated[260])/(data_treated["high_fit"] - data_treated["low_fit"])
data_treated["normed_data"] = 1- ana.normalize(raw).loc[260].to_numpy()
data_treated["log_fit"] = 1 - ana.normalize(data_treated)["fit"]
plot.mult_func([260, "low_fit", "high_fit", "intermediate", "fit"], [data_treated.T], subtitle=[""], title="Hypochromatic shift with Plaetau Fits", y_label="Absorbance at 260nm")
plot.mult_func(["baseline_fit", "normed_data", "log_fit"], [data_treated.T], subtitle=[""],  title="Fraction folded calculated by Plateu fits and Normalization", y_label="Fraction folded")


**calulate K and ln K for different methods**

In [ ]:
c_total = 40 * (10**-9) # # total strand concentration in molar

# get K and ln K
base_method = pd.DataFrame(data_treated["baseline_fit"].to_numpy(), index=[20,30,40,45,50,56,56,58,60,62,64,67,70,75,80, 90], columns=["theta"])
base_method['theta'] = base_method['theta'].astype(float)

# here equation depending on reaction type, choosen: Bimolecular, complementary strand types
base_method["K"] = (base_method["theta"])/(c_total*((1 - base_method["theta"])**2))
base_method['ln K'] = np.log(base_method['K'])

# select values in confidence values 0.1 < theta < 0.9
base_method  = base_method.loc[60:70]
# change celsius to kelvin
base_method = base_method.set_index(1/(base_method.index + 273.15))

# make linear fit and plot
enthalpy_base, params_base = ana.lm_fit(base_method.T, wave="ln K", f_type='linear', guess=[80000, -2])
error = {"fit" : enthalpy_base["error"].to_numpy()}
plot.mult_func(["ln K", "fit"], [enthalpy_base.T], error=error, x_label="1/T [1/Kelvin]", y_label="ln K")

In [ ]:
c_total = 40 * (10**-9) # # total strand concentration in molar

# get K and ln K
normed_method = pd.DataFrame(data_treated["normed_data"].to_numpy(), index=[20,30,40,45,50,56,56,58,60,62,64,67,70,75,80, 90], columns=["theta"])
normed_method['theta'] = normed_method['theta'].astype(float)

# here equation depending on reaction type, choosen: Bimolecular, complementary strand types
normed_method["K"] = (normed_method["theta"])/(c_total*((1 - normed_method["theta"])**2))
normed_method['ln K'] = np.log(normed_method['K'])

# select values in confidence values 0.1 < theta < 0.9
normed_method  = normed_method.loc[60:70]
# change celsius to kelvin
normed_method = normed_method.set_index(1/(normed_method.index + 273.15))

# make linear fit and plot
enthalpy_normed, params_norm = ana.lm_fit(base_method.T, wave="ln K", f_type='linear', guess=[80000, -2])
error = {"fit" : enthalpy_normed["error"].to_numpy()}
plot.mult_func(["ln K", "fit"], [enthalpy_normed.T], error=error, x_label="1/T [1/Kelvin]", y_label="ln K")

In [ ]:
c_total = 40 * (10**-9) # # total strand concentration in molar

# get K and ln K
fit_method = pd.DataFrame(data_treated["log_fit"].to_numpy(), index=[20,30,40,45,50,56,56,58,60,62,64,67,70,75,80, 90], columns=["theta"])
fit_method['theta'] = fit_method['theta'].astype(float)

# here equation depending on reaction type, choosen: Bimolecular, complementary strand types
fit_method["K"] = (fit_method["theta"])/(c_total*((1 - fit_method["theta"])**2))
fit_method['ln K'] = np.log(fit_method['K'])

# select values in confidence values 0.1 < theta < 0.9
fit_method  = fit_method.loc[60:70]
# change celsius to kelvin
fit_method = fit_method.set_index(1/(fit_method.index + 273.15))

# make linear fit and plot
enthalpy_fit, params_fit = ana.lm_fit(fit_method.T, wave="ln K", f_type='linear', guess=[80000, -2])
error = {"fit" : enthalpy_fit["error"].to_numpy()}
plot.mult_func(["ln K", "fit"], [enthalpy_fit.T], error=error, x_label="1/T [1/Kelvin]", y_label="ln K")

**Calculate Entropy and Enthalpy**

In [ ]:
enthalpy = pd.DataFrame(["H [kJ/mol]", "H [kcal/mol]", "S [kJ/mol*K]", "S [kcal/mol*K]"], columns=["units"])
enthalpy = enthalpy.set_index("units")

# for base method
m, y0 = params_base.values()
delta_H = -m * 8.3145
delta_S = y0 * 8.3145
enthalpy["base"] = [delta_H*10**(-3), delta_H*0.2390057*10**(-3), delta_S*10**(-3), delta_S*0.2390057*10**(-3)]

# for normed method
m, y0 = params_norm.values()
delta_H = -m * 8.3145
delta_S = y0 * 8.3145
enthalpy["norm"] = [delta_H*10**(-3), delta_H*0.2390057*10**(-3), delta_S*10**(-3), delta_S*0.2390057*10**(-3)]

# for fit method
m, y0 = params_fit.values()
delta_H = -m * 8.3145
delta_S = y0 * 8.3145
enthalpy["fit"] = [delta_H*10**(-3), delta_H*0.2390057*10**(-3), delta_S*10**(-3), delta_S*0.2390057*10**(-3)]

print(enthalpy)

## Derivative


In [ ]:
deriv = ana.derivative(data_all["Control_0M"].absorb_df)
data = data_all["Control_0M"].absorb_df

In [ ]:
plot.mult_func([260], [data], [deriv], title= "Hypochromatic shift", subtitle=["Absolute value", "interpolated derivative"], label=[""], y_label="Absorbance")

# Calibration 

In [29]:
datapath = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Data\\Calibration"
datalist = os.listdir(datapath)
# 0 for PL, 1 for CSA
filename = os.path.join(datapath, datalist[0])

print(filename)

C:\Users\crazy\Documents\Uni\Masterarbeit\Projekt\Data\Calibration\Calibration Measurement 11.5 mM PL-F_Temperature25.00.txt


In [30]:
file = open(filename, "r")
list_of_lines = file.readlines()

# Parameter for head and tail part of document which will be deleted
head = 21
tail = 1572
del list_of_lines[tail:]
del list_of_lines[:head]

# separating numbers into sublist
for i in range(len(list_of_lines)):
    list_of_lines[i] = list_of_lines[i].replace("\n", "")
    list_of_lines[i] = list_of_lines[i].replace(",", ".")
    list_of_lines[i] = list_of_lines[i].split('\t', 3)

# transform list of lists to array of float-type
matrix = np.array(list_of_lines)
matrix = matrix.astype(np.float)
file.close()

calibration_data = pd.DataFrame(matrix, columns=['wavelength', 'CD Values', 'HT Values', 'Absorbance'])
calibration_data = calibration_data.set_index('wavelength')

In [31]:
plot.mult_func(['CD Values'], [calibration_data], swap=True)
plot.mult_func(['Absorbance'], [calibration_data], swap=True)
plot.mult_func(['HT Values'], [calibration_data], swap=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Molar Ellipticity**

In [32]:
# for PL
molarity = calibration_data.loc[215,'Absorbance']  * 10**4 / 90 # molarity in mMs
print('Determined concentration in mmol/l: \t', molarity)
pathlength =  0.1 # cm 
print('Ratio experimental vs. theoretical value at the maximum around 290 nm: \n with determinded molarity: ',
      calibration_data.loc[calibration_data['CD Values'].idxmin(axis=0), 'CD Values']  / (0.001* molarity) / (-4.9*3300),
      '\t  with wanted molarity 11.53 mM:' , calibration_data.loc[calibration_data['CD Values'].idxmin(axis=0), 'CD Values']  / (0.001* 11.53) / (-4.9*3300) ) 
      

Determined concentration in mmol/l: 	 9.286277777777778
Ratio experimental vs. theoretical value at the maximum around 290 nm: 
 with determinded molarity:  1.0692125054415886 	  with wanted molarity 11.53 mM: 0.8611452150047119


In [27]:
# for CSA
molarity = calibration_data.loc[285,'Absorbance'] *  (10**4) / 34.5  # molarity in mMs
print('Determined concentration in mmol/l: \t', molarity)
pathlength =  0.1 # cm 
print('Ratio experimental vs. theoretical value at the maximum around 290 nm: \n with determinded molarity: ',
      calibration_data.loc[calibration_data['CD Values'].idxmax(axis=0), 'CD Values']  / (0.001* molarity) / (7800),  
      '\t  with wanted molarity 4.7 mM:' , calibration_data.loc[calibration_data['CD Values'].idxmax(axis=0), 'CD Values']  / (0.001* 4.7) / (7800)  ) 
print('Ratio experimental vs. theoretical value at the minimum around 192.5 nm: \n with determinded molarity: ',
      calibration_data.loc[calibration_data['CD Values'].idxmin(axis=0), 'CD Values']  / (0.001* molarity) / (-15600 ),  
      '\t  with wanted molarity 4.7 mM:' , calibration_data.loc[calibration_data['CD Values'].idxmin(axis=0), 'CD Values']  / (0.001* 4.7) / (-15600 ) )
print('Ratio between absorbance 285 nm and molar ellipticity at 290.5 nm: ', 
      (calibration_data.loc[calibration_data['CD Values'].idxmax(axis=0), 'CD Values']  / (3300)) / calibration_data.loc[285, 'Absorbance'],
     ' -> this value should be 0.068') # should be 0.068
 

Determined concentration in mmol/l: 	 5.473130434782609
Ratio experimental vs. theoretical value at the maximum around 290 nm: 
 with determinded molarity:  0.7917748774900709 	  with wanted molarity 4.7 mM: 0.9220185488270595
Ratio experimental vs. theoretical value at the minimum around 192.5 nm: 
 with determinded molarity:  0.812555887707617 	  with wanted molarity 4.7 mM: 0.9462179487179487
Ratio between absorbance 285 nm and molar ellipticity at 290.5 nm:  0.5424544615215243  -> this value should be 0.068


# Test

**Derivative**

In [8]:
keys = ['Control_0M', 'GdmCl_4M', 'Gdm2SO4_2M']

In [9]:
for key in keys:
    df = data_all[key].cd_df
    data = df.to_numpy()
    t_vec = df.columns.to_numpy()
    delta_x = data[:, 2:] - data[:, :-2]
    delta_t = t_vec[2:] - t_vec[:-2]
    deriv =  pd.DataFrame(np.divide(delta_x, delta_t), index=df.index, columns=df.loc[:, 30:80].columns)
    deriv = deriv.astype('float64')
    plot.heatmap(deriv.loc[215:300], c_min=[-0.8e7], c_max=[0.8e7], subtitle=[key], title='')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Concentration calculation**

In [58]:
df = pd.DataFrame()
liste = ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M', 'Gdm2SO4_2M', 'Gdm2SO4_4M','GdmCl_0.5M', 'GdmCl_1M', 'GdmCl_2M', 'GdmCl_4M']
for key in liste:
    df[key] = data_all[key].cd_df[70]
    
plot.mult_func(liste, [df], swap=True, x_min=215, x_max=300, subtitle=['Data at 70°C'], x_label='Wavelengt / nm' )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
for key in data_all.keys():
    absorb_coeff = 0.0263 #ml/µg
    conc = data_all[key].absorb_df.loc[260,20] / (absorb_coeff* 0.1) # µg/ml
    mol_weight = 4472760.4 *(10**(6)) # µg/mol
    conc = conc / mol_weight #mol/ml
    conc =  conc * (10**(3)) # mol/L = M
    conc = conc * (10**(9)) # nmol/L = nM
    
    conc2 = data_all[key].absorb_df.loc[260,20] / (0.02* 0.1) # µg/ml
    mol_weight = 4472760.4 *(10**(6)) # µg/mol
    conc2 = conc2 / mol_weight #mol/ml
    conc2 =  conc2 * (10**(3)) # mol/L = M
    conc2 = conc2 * (10**(9)) # nmol/L = nM

    print(key, round(conc,3), round(conc2, 3), round(conc-conc2,3 ))

46_forward 24.757 32.556 -7.798
46_reversed 25.974 34.156 -8.182
Control_0M 30.354 39.915 -9.561
Gdm2SO4_0.5M 32.511 42.752 -10.241
Gdm2SO4_1M 31.293 41.151 -9.857
Gdm2SO4_2M 33.782 44.423 -10.641
Gdm2SO4_4M 34.202 44.975 -10.774
Gdm2SO4_6M 30.511 40.122 -9.611
GdmCl_0.5M 32.273 42.439 -10.166
GdmCl_1M 28.71 37.754 -9.044
GdmCl_2M 29.204 38.403 -9.199
GdmCl_2M_20nM Origami 16.14 21.224 -5.084
GdmCl_2M_24h 32.597 42.865 -10.268
GdmCl_2M_80nM Origami 71.692 94.275 -22.583
GdmCl_4M 34.448 45.299 -10.851
GdmCl_6M 30.839 40.554 -9.714
GdmCl_6M_alt 27.26 35.847 -8.587
GdmSCN_0.5M 32.171 42.305 -10.134
GdmSCN_2M 36.621 48.156 -11.536
Urea_2M 30.79 40.489 -9.699


In [23]:
print_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Auswertung"

In [25]:
excel_path = print_path + "\\" + "\\output.xlsx"
print(excel_path)

C:\Users\crazy\Documents\Uni\Masterarbeit\Projekt\Auswertung\\output.xlsx


In [26]:
liste = ['Control_0M', 'GdmCl_1M', 'GdmCl_2M', 'GdmCl_4M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M', 'Gdm2SO4_2M']
for key in liste:
    df = data_all[key].absorb_df.loc[260]
    with pd.ExcelWriter(excel_path, mode='a') as writer:  
        df.round(3).to_excel(writer, sheet_name=key)

**Simulation of Melting Curves**

In [30]:
# gas constant
r =  8.314

# first process -> melting dsDNA to ssDNA in tensed state
t_01 =  337
dh_1 = 123000

# second process -> melting dsDNA to ssDNA in relaxed state
t_02 = 335.9375
dh_2 = 215000

# third process -> relaxing origami in dsDNA
t_03 = 323.14502
dh_3 = 10000 

# add dc_p values
dc_1 = 2000
dc_2 = 4000
dc_3 = 1000

# temperature points and dt to reference temperature t_01
t =  np.arange(273, 372, 1)

# create lists to save calcua
rel_conc = pd.DataFrame(index=t, columns=[1,2,3,4])

In [31]:
# calculate fractions for all temperature points
for i in t:
    # calculate dg
    dg_1 = - (dh_1/t_01 - dc_1) * (i - t_01) + dc_1 * i * np.log(t_01/i) # dsDNA to ssDNA in tensed state 
    dg_2 = - (dh_2/t_01 - dc_2) * (i - t_02) + dc_2 * i * np.log(t_02/i) # dsDNA to ssDNA in relaxed state
    dg_3 = - (dh_3/t_01 - dc_3) * (i - t_03) + dc_3 * i * np.log(t_03/i) # tensed state to relaxed state in dsDNA
    
    # calculate equilibrium constants k
    k_1 = np.exp( - dg_1 / (r * i))
    k_2 = np.exp( - dg_2 / (r * i))
    k_3 = np.exp( - dg_3 / (r * i))
    
    # calculate relative concentrations
    sum_of_states = 1 + k_1 + k_3 + k_3 * k_2 # number of states / number of state 1 
    rel_conc.loc[i, 1] = 1 / sum_of_states # norm.population of dsDNA tensed 
    rel_conc.loc[i, 2] = k_1 /sum_of_states # norm.population of ssDNA tensed
    rel_conc.loc[i, 3] = k_3 /sum_of_states # norm.population of dsDNA relaxed
    rel_conc.loc[i, 4] = k_3 * k_2 /sum_of_states # norm.population of ssDNA relaxed 


In [32]:
plot.mult_func([1,2,3,4], [rel_conc.T])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Beers Law**

In [19]:
for key in data_all.keys():
    absorbance = data_all[key].absorb_df.loc[260, 20]
    df = data_all[key].cd_df
    
    conc_ds =  absorbance/(0.02*0.1) #[µg/mL]
    molar = conc_ds * 10**(-6) / (4472760.4 * 10**(-3)) * (10**9) #[nmol/l] 
    df_norm = 100 * df / (molar * 0.1)    
    print(key, ":  \t", molar , 'nM')

46_forward :  	 32.555510909996435 nM
46_reversed :  	 34.156088486206414 nM
Control_0M :  	 39.91528363558218 nM
Gdm2SO4_0.5M :  	 42.751787017252255 nM
Gdm2SO4_1M :  	 41.15076228988254 nM
Gdm2SO4_2M :  	 44.423461627857364 nM
Gdm2SO4_4M :  	 44.97535794673911 nM
Gdm2SO4_6M :  	 40.121643895791955 nM
GdmCl_0.5M :  	 42.43866941765983 nM
GdmCl_1M :  	 37.753754929506165 nM
GdmCl_2M :  	 38.402794837836595 nM
GdmCl_2M_20nM Origami :  	 21.22391800821703 nM
GdmCl_2M_24h :  	 42.86480447287093 nM
GdmCl_2M_80nM Origami :  	 94.27466760794967 nM
GdmCl_4M :  	 45.29876002300503 nM
GdmCl_6M :  	 40.55392727944917 nM
GdmCl_6M_alt :  	 35.84687880888947 nM
GdmSCN_0.5M :  	 42.30485943311428 nM
GdmSCN_2M :  	 48.15627950918185 nM
Urea_2M :  	 40.48909036129009 nM


In [ ]:
for key in data_all.keys():
    plot.mult_func([280], [data_all[key].cd_df], subtitle=[key], y_scaling=True, y_max=[3.85], y_min=[1.75])

In [ ]:


        # put second y_axis for "GdmCl_0.5M" normed
        # function to bring back values
        def expand_back(y):
            return y * (0.523893 - 0.379531) + 0.379531
        # inverse function
        def norm(y):
            return (y - 0.379531) / (0.523893 - 0.379531)

        # put axis and label
        secax = ax.secondary_yaxis('right', functions=(expand_back, norm), fontsize=16)
        secax.set_ylabel('Absorption [%]', fontsize=16)

In [ ]:
df = data_all["Control_0M"].cd_df
mean = df.mean(axis=1)
var =  df.var(axis=1)
dyn = df.subtract(mean, axis=0)

col = dyn.columns 
idx = dyn.index
m = len(col)
n =len(idx)

In [ ]:
wave1 =  247
wave2 = 260
i = 20

print(np.sum((dyn.loc[wave1, :]/mean[wave1] - dyn.loc[wave2, :]/mean[wave2]) * col) * np.sqrt(var.loc[wave1]*var.loc[wave2])/(m*(m-1)) )


In [ ]:

assync = pd.DataFrame(np.zeros((n,n)), index=idx, columns=idx)

for j in idx:
    for k in idx:
        assync.loc[j,k] = np.sum((dyn.loc[j, :]/mean[j] - dyn.loc[k, :]/mean[k]) * col) * np.sqrt(var.loc[j]*var.loc[k])/(m*(m-1))
print(assync)

In [ ]:
plot.heatmap(assync, x_min=[220], y_min=[220])

In [9]:
# read by default 1st sheet of an excel file
print_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Auswertung\\"
data_path1 = os.path.join(print_path, 'AveragedSpectra.xlsx')
df= pd.read_excel(data_path1)
df= df.drop(0, axis=0)
df = df.set_index('Wavelength')
plot.mult_func(['S1', 'S2', 'S2\'', 'S3'], [df.T], marker=['','','',''], subtitle=[''], x_label='wavelength / nm',
              colors=['tab:red', 'tab:blue', 'tab:grey', 'tab:green'], y_label='relative ellipticity', baseline=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Parafak Ethanol

## *setting up the Notebook*

In [ ]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget
        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import Output
import matplotlib

from scipy import integrate
import lmfit
import os

In [ ]:
import analise as ana
import cdata 
import hotznplots as plot

<div class="alert alert-block alert-info">
<b>Info:</b> Print always whole DataFrames
</div>

In [ ]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [ ]:
# change plot size
plt.rcParams["figure.figsize"] = (11,9)

<div class="alert alert-block alert-info">
<b>Info:</b> Get the data
</div>

In [ ]:
# Path for Daniel:
#path = "F:\\HZDR\\CD_data"

# My Path:
path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\Data\\CD_data\\Ethanol"
datalist = os.listdir(path)
print(datalist)

In [ ]:
# all data in one dictionary with foldernames as names
data_all = {}
for i in range(len(datalist)):
    data_all[datalist[i]] = cdata.CData(os.path.join(path, datalist[i]))

In [ ]:
df =  data_all["Control_0%"].cd_df
df = df.rename(columns={-1:"0%"})
df["50%"] =  (data_all["Ethanol_50%"].cd_df.loc[:,-1])
df["nach"] =  (data_all["Ethanol_50%_nachTest"].cd_df.loc[:,-1])
df["vor"] =  (data_all["Ethanol_50%_vorTest"].cd_df.loc[:,-1])
df["50% real"] =  (data_all["Ethanol_50%_real"].cd_df.loc[:,-1])
print(df)

In [ ]:
plot.mult_func(['0%', 'vor', 'nach', '50%', "50% real" ], [df], swap=True, marker=["", "", "", "", ""], x_label="Wavelengt [nm]")

# Salmon testes

## setting up the notebook


In [ ]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget
        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import Output
import matplotlib
import openpyxl

from scipy import integrate
import lmfit
import os

In [ ]:
import analise as ana
import cdata 
import hotznplots as plot

<div class="alert alert-block alert-info">
<b>Info:</b> Print always whole DataFrames
</div>

In [ ]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [ ]:
# change plot size
plt.rcParams["figure.figsize"] = (10,7)

<div class="alert alert-block alert-info">
<b>Info:</b> Get the data
</div>

In [ ]:
# Path for Daniel:
#path = "F:\\HZDR\\CD_data"

# My Path:
path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\Data\\CD_data\\Salmon Testes"
datalist_salmon = os.listdir(path)
print(datalist_salmon)

In [ ]:
# all data in one dictionary with foldernames as names
data_salmon = {}
for i in range(len(datalist_salmon)):
    data_salmon[datalist_salmon[i]] = cdata.CData(os.path.join(path, datalist_salmon[i]))

In [ ]:
for key in data_salmon.keys():
    absorbance = data_salmon[key].absorb_df.loc[260, 20]
    df = data_salmon[key].cd_df
    
    conc_ds =  absorbance/(0.02*0.1*(10**3)) #[mg/mL]
    print(key, ":  \t", conc_ds)

In [ ]:
for key in data_salmon.keys():
    plot.mult_func([260], [data_salmon[key].absorb_df], subtitle=[key])

In [ ]:
print(data_salmon["Control_0M_continous"].t_list)

In [ ]:
plot.mult_func([20, 40, 60, 80], [data_salmon['Control_0M_continous'].cd_df], swap=True, baseline=True, marker=["","","",""], subtitle=["Salmon Control"], linestyle=["-",":", ":", "-"], x_min=220)